# Predicting NBA Scores using Machine Learning

### George Tao,  Jeremiah Sahabu, Isaac Chau

#### Introduction

Sports inheritantly has large variability. Thus, we wanted to see if we can predict which team will win through supervised Machine Learning. We decided to use Logistic Regression for our models as we believe it fits the scenario best.

#### Dataset:

#### Features:

We saw a lot of features given ranging from player stats, scores, among others. We used a combination of these features as well as features we created from the dataset above to feed into our model. See list below of features we identified significant

- Consecutive Away Games
- Wins in the last 10 Games
- Officials 
- Players
- Back to Back Games
- eFG% (season average)
- eFG% allowed (season average)
- Team Fouls (season average)
- Turnovers 
- Offensive Rebounds



### Libraries

In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
from statistics import median

# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np


# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report
import nltk
import urllib.request as urllib2 # For python3
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture as GMM
from matplotlib.colors import LogNorm

### Loading in the Data

### Parsing the Data

In [1]:
#Storing DataFrames in Separate Dicts
team_names = list(teamBox.teamAbbr.unique())
team_dict = { i : 0 for i in team_names }

NameError: name 'teamBox' is not defined

In [ ]:
#Parsing Data and Finding Consequetive Away Games
teams = teamBox[["teamAbbr", "teamAbbr", "teamLoc", "teamRslt", "gmDate"]]
teams.columns = ["groups", "teamAbbr", "teamLoc", "teamRslt", "gmDate"]

#Grouping Data by Team
sorted_teams = teams.groupby('groups')
sorted_teams.first()

#Finding Consequetive Away Games
for teams in team_names:
    temp = sorted_teams.get_group(teams)
    temp = temp.reset_index()
    Counter = 0
    for ind, row in temp.iterrows():
        if row['teamLoc'] == "Away":
            Counter += 1
            temp.loc[ind, "Consequetive Away Games"] = Counter
        else:
            Counter = 0
            temp.loc[ind, "Consequetive Away Games"] = Counter
    team_dict[teams] = temp
    
#Finding Number of Wins in Past 10 Games

    for ind in temp.index:
        if ind < 10:
            Counter = 0
            for x in range(ind):
                if temp['teamRslt'][x] == "Win":
                    Counter += 1
            temp.loc[ind, "Wins in Last 10"] = Counter
        else:
            Counter = 0
            for x in range(ind-10,ind):
                if temp['teamRslt'][x] == "Win":
                    Counter += 1
            temp.loc[ind, "Wins in Last 10"] = Counter

#Combining the Tables 
final_1 = pd.concat([team_dict['ATL'], team_dict['BKN'], team_dict['BOS'], team_dict['CHA'], team_dict['CHI'], team_dict['CLE'], team_dict['DAL'], team_dict['DEN'], team_dict['DET'], team_dict['GS']], ignore_index=True)
final_2 = pd.concat([team_dict['HOU'], team_dict['IND'], team_dict['LAC'], team_dict['LAL'], team_dict['MEM'], team_dict['MIA'], team_dict['MIL'], team_dict['MIN'], team_dict['NO'], team_dict['NY']], ignore_index=True)
final_3 = pd.concat([team_dict['OKC'], team_dict['ORL'], team_dict['PHI'], team_dict['PHO'], team_dict['POR'], team_dict['SA'], team_dict['SAC'], team_dict['TOR'], team_dict['UTA'], team_dict['WAS']], ignore_index=True)

final = pd.concat([final_1, final_2, final_3], ignore_index=True)

final = final.sort_values('index')
final = final.set_index('index')

In [ ]:
#Color Scheme

for ind, row in final.iterrows():
    if row["teamRslt"] == "Win":
        final.loc[ind, "Color Scheme"] = 0
    else:
        final.loc[ind, "Color Scheme"] = 1

### Models & Plots

### Conclusion